In [2]:
#!/usr/bin/env python3
# Robust EASE-Grid 2.0 sampling (EPSG:6933) for SMAP L4 soil moisture (North-hemisphere subset in your files)
# - Reprojects sites to EPSG:6933 (EASE-Grid 2.0 Global, 9 km)
# - Computes (row, col) from a 90°N anchor (no hard-coded XMIN/YMAX)
# - Auto-detects subset height from the data (e.g., 40N+ => 289 rows)
# - Indexes sm_surface / sm_rootzone directly
# - Writes monthly CSV for 2000–2023

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from pyproj import Transformer

# -------------------- CONFIG --------------------
SHAPEFILE   = "/explore/nobackup/people/spotter5/anna_v/v2/v2_model_sites.shp"
H5_DIR      = "/explore/nobackup/people/spotter5/anna_v/v2/L4_SM_NRv11-4_40N+_soil_moisture"
OUT_CSV     = "/explore/nobackup/people/spotter5/anna_v/v2/soil_moisture_by_site_monthly_2000_2023.csv"

START_YEAR  = 2000
END_YEAR    = 2023

# EASE-Grid 2.0 Global (EPSG:6933) constants for 9-km product
PIX_SIZE_M    = 9000.0          # meters
GLOBAL_NCOLS  = 3856            # confirmed global width
FILLVALUE     = -9999.0
# ------------------------------------------------

def build_filename(year: int, month: int) -> str:
    return f"L4_SM_NRv11-4_40N+_soil_moisture_Y{year:04d}M{month:02d}.h5"

def read_arrays(h5_path: str):
    """Open H5 and return (sm_surface, sm_rootzone) as float32 numpy arrays with FILLVALUE set to NaN."""
    ds = xr.open_dataset(h5_path, engine="h5netcdf", phony_dims="sort")
    sm_surf = ds["sm_surface"].values.astype("float32")
    sm_root = ds["sm_rootzone"].values.astype("float32")
    sm_surf[sm_surf == FILLVALUE] = np.nan
    sm_root[sm_root == FILLVALUE] = np.nan
    # Mild sanity: width should match global columns
    nrows, ncols = sm_surf.shape
    if ncols != GLOBAL_NCOLS:
        raise ValueError(f"Unexpected number of columns {ncols} in {os.path.basename(h5_path)}; expected {GLOBAL_NCOLS}.")
    return sm_surf, sm_root

def make_transformer():
    # lon/lat -> EPSG:6933
    return Transformer.from_crs("EPSG:4326", "EPSG:6933", always_xy=True)

def compute_row_anchor(transformer):
    """
    Compute projected Y at 90N to anchor the TOP of the north-hemisphere grid.
    Row index is measured down from this anchor in PIX_SIZE_M increments.
    """
    _, y90 = transformer.transform(0.0, 90.0)
    return y90  # row 0 at 90N

def lonlat_to_epsg6933(transformer, lon, lat):
    return transformer.transform(lon, lat)

def xy_to_colrow_from_anchors(x_m, y_m, y_top_anchor):
    """
    Column (centered): col = floor(x/PIX + GLOBAL_NCOLS/2)
    Row anchored to y at 90N: row_global_from_90 = floor((y_top_anchor - y)/PIX)
    For any north-hemisphere subset starting at 90N, local rows are the same as row_global_from_90,
    and validity is determined by checking within the current array's (nrows, ncols).
    """
    col = int(np.floor(x_m / PIX_SIZE_M + GLOBAL_NCOLS / 2.0))
    row_from_90 = int(np.floor((y_top_anchor - y_m) / PIX_SIZE_M))
    return col, row_from_90

def find_first_existing_file():
    """Locate the first existing monthly file to report subset shape for logging."""
    for year in range(START_YEAR, END_YEAR + 1):
        for month in range(1, 13):
            fpath = os.path.join(H5_DIR, build_filename(year, month))
            if os.path.exists(fpath):
                try:
                    sm_surface, _ = read_arrays(fpath)
                    return fpath, sm_surface.shape
                except Exception:
                    # Keep looking if a particular month is corrupt
                    pass
    return None, None

def main():
    # Load sites; keep only rows with geometry
    gdf = gpd.read_file(SHAPEFILE)
    gdf = gdf[gdf.geometry.notnull()].copy()
    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326")
    else:
        gdf = gdf.to_crs("EPSG:4326")

    # Normalize site id column
    if "site_refer" in gdf.columns and "site_reference" not in gdf.columns:
        gdf = gdf.rename(columns={"site_refer": "site_reference"})
    if "site_reference" not in gdf.columns:
        gdf["site_reference"] = [f"site_{i}" for i in range(len(gdf))]

    # Prepare transformer and compute top (90N) y-anchor
    tf = make_transformer()
    y_top = compute_row_anchor(tf)

    # Precompute site indices (no lat-cut; bounds checked later per file)
    site_index_info = []
    for _, row in gdf.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        x_m, y_m = lonlat_to_epsg6933(tf, lon, lat)
        col, row_from_90 = xy_to_colrow_from_anchors(x_m, y_m, y_top)
        site_index_info.append({
            "site_reference": row["site_reference"],
            "col": col,
            "row_from_90": row_from_90
        })

    site_idx_df = pd.DataFrame(site_index_info)
    # Ensure numeric dtype
    site_idx_df["row_from_90"] = pd.to_numeric(site_idx_df["row_from_90"], errors="coerce")
    site_idx_df["col"]         = pd.to_numeric(site_idx_df["col"], errors="coerce")

    # Log the subset shape (from first available file)
    first_file, first_shape = find_first_existing_file()
    if first_file and first_shape:
        nrows0, ncols0 = first_shape
        print(f"[INFO] First available file: {os.path.basename(first_file)} has shape {first_shape} "
              f"(implies subset height {nrows0} rows from 90°N).")
    else:
        print("[WARN] Could not locate any input files to infer subset shape; proceeding anyway.")

    records = []

    for year in range(START_YEAR, END_YEAR + 1):
        for month in range(1, 13):
            fpath = os.path.join(H5_DIR, build_filename(year, month))
            if not os.path.exists(fpath):
                print(f"[WARN] Missing file: {fpath}")
                continue

            try:
                sm_surface, sm_rootzone = read_arrays(fpath)
            except Exception as e:
                print(f"[ERROR] {fpath}: {e}")
                continue

            nrows, ncols = sm_surface.shape  # subset rows may be 289 for 40N+, etc.

            # Sample each site using direct numpy indexing
            for _, r in site_idx_df.iterrows():
                rs = int(r["row_from_90"]) if pd.notna(r["row_from_90"]) else -1
                cs = int(r["col"])         if pd.notna(r["col"])         else -1

                if (0 <= rs < nrows) and (0 <= cs < ncols):
                    val_s = sm_surface[rs, cs]
                    val_r = sm_rootzone[rs, cs]
                else:
                    val_s = np.nan
                    val_r = np.nan

                records.append({
                    "site_reference": r["site_reference"],
                    "year": year,
                    "month": month,
                    "sm_surface": float(val_s) if np.isfinite(val_s) else np.nan,
                    "sm_rootzone": float(val_r) if np.isfinite(val_r) else np.nan
                })

    out_df = pd.DataFrame.from_records(records)
    out_df.sort_values(["site_reference", "year", "month"], inplace=True)
    out_df.to_csv(OUT_CSV, index=False)
    print(f"Saved: {OUT_CSV}  (rows={len(out_df)})")

if __name__ == "__main__":
    main()


[INFO] First available file: L4_SM_NRv11-4_40N+_soil_moisture_Y2000M01.h5 has shape (289, 3856) (implies subset height 289 rows from 90°N).
Saved: /explore/nobackup/people/spotter5/anna_v/v2/soil_moisture_by_site_monthly_2000_2023.csv  (rows=616032)


In [16]:
df = pd.read_csv('/explore/nobackup/people/spotter5/anna_v/v2/soil_moisture_by_site_monthly_2000_2023.csv')

df

,site_reference,year,month,sm_surface,sm_rootzone
0,APEX Beta_Active Margin_AMCH1_agg_chamber,2000,1,0.538579,0.848821
1,APEX Beta_Active Margin_AMCH1_agg_chamber,2000,2,0.538579,0.848813
2,APEX Beta_Active Margin_AMCH1_agg_chamber,2000,3,0.538579,0.848805
3,APEX Beta_Active Margin_AMCH1_agg_chamber,2000,4,0.639189,0.881813
4,APEX Beta_Active Margin_AMCH1_agg_chamber,2000,5,0.660515,0.897082
...,...,...,...,...,...
617467,Zotino_RU-Zot_tower,2023,8,0.226507,0.151379
617468,Zotino_RU-Zot_tower,2023,9,0.208362,0.137487
617469,Zotino_RU-Zot_tower,2023,10,0.234460,0.144890
617470,Zotino_RU-Zot_tower,2023,11,0.238286,0.152914


In [13]:
df['sm_surface'].unique()

array([nan])